In [2]:
pip install lyricsgenius pandas


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install spotipy


Note: you may need to restart the kernel to use updated packages.


In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
import pandas as pd
import time

# Spotify API credentials
spotify_client_id = ''
spotify_client_secret = ''

# Genius API credentials
genius_access_token = ''

# Spotify API setup
client_credentials_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Genius API setup
genius = lyricsgenius.Genius(genius_access_token)

def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

def get_lyrics(song_title, artist_name):
    try:
        song = genius.search_song(song_title, artist_name)
        if song:
            return song.lyrics
    except:
        print(f"Error fetching lyrics for {song_title} by {artist_name}")
    return "Lyrics not found"

def get_artist_genres(artist_id):
    artist = sp.artist(artist_id)
    return artist['genres']

def get_audio_features(track_id):
    features = sp.audio_features(track_id)[0]
    track = sp.track(track_id)
    features['duration_ms'] = track['duration_ms']
    return features

def get_album_info(album_id):
    album = sp.album(album_id)
    return album['name'], album['release_date'], album['available_markets']

def format_duration(ms):
    seconds = ms // 1000
    minutes, seconds = divmod(seconds, 60)
    return f"{minutes}:{seconds:02d}"

def main(playlist_id):
    tracks = get_playlist_tracks(playlist_id)
    data = []
    
    for i, track in enumerate(tracks, 1):
        song_title = track['track']['name']
        artist_name = track['track']['artists'][0]['name']
        artist_id = track['track']['artists'][0]['id']
        track_id = track['track']['id']
        album_id = track['track']['album']['id']
        isrc = track['track'].get('external_ids', {}).get('isrc', 'Not available')
        print(f"Processing song {i}/{len(tracks)}: {song_title} by {artist_name}")
        
        lyrics = get_lyrics(song_title, artist_name)
        genres = get_artist_genres(artist_id)
        genre = genres[0] if genres else "Unknown"
        audio_features = get_audio_features(track_id)
        album_name, release_date, available_markets = get_album_info(album_id)
        
        data.append({
            'Title': song_title,
            'Artist': artist_name,
            'Album': album_name,
            'Release_Date': release_date,
            'Available_Markets': ', '.join(available_markets),
            'Genre': genre,
            'ISRC': isrc,
            'Lyrics': lyrics,
            'Tempo': audio_features['tempo'],
            'Key': audio_features['key'],
            'Mode': audio_features['mode'],
            'Danceability': audio_features['danceability'],
            'Energy': audio_features['energy'],
            'Instrumentalness': audio_features['instrumentalness'],
            'Acousticness': audio_features['acousticness'],
            'Speechiness': audio_features['speechiness'],
            'Valence': audio_features['valence'],
            'Loudness': audio_features['loudness'],
            'Track_Popularity': track['track']['popularity'],
            'Duration_ms': audio_features['duration_ms'],
            'Duration': format_duration(audio_features['duration_ms'])
        })
        
        time.sleep(3)
    
    df = pd.DataFrame(data)
    df.to_csv('playlist_data.csv', index=False)
    print("Data collection complete. Check 'playlist_data.csv' for results.")

# Replace with your playlist ID
playlist_id = ''
main(playlist_id)


Processing song 1/245: High For This by The Weeknd
Searching for "High For This" by The Weeknd...
Done.
Processing song 2/245: What You Need by The Weeknd
Searching for "What You Need" by The Weeknd...
Done.
Processing song 3/245: House Of Balloons / Glass Table Girls by The Weeknd
Searching for "House Of Balloons / Glass Table Girls" by The Weeknd...
Done.
Processing song 4/245: The Morning by The Weeknd
Searching for "The Morning" by The Weeknd...
Done.
Processing song 5/245: Wicked Games by The Weeknd
Searching for "Wicked Games" by The Weeknd...
Done.
Processing song 6/245: The Party & The After Party by The Weeknd
Searching for "The Party & The After Party" by The Weeknd...
Done.
Processing song 7/245: Coming Down by The Weeknd
Searching for "Coming Down" by The Weeknd...
Done.
Processing song 8/245: Loft Music by The Weeknd
Searching for "Loft Music" by The Weeknd...
Done.
Processing song 9/245: The Knowing by The Weeknd
Searching for "The Knowing" by The Weeknd...
Done.
Processi